In [8]:
%load_ext autoreload
%autoreload 2

import os
os.chdir('/Users/mhendriksen/Desktop/repositories/evaluating-cmr-in-mm/')

In [71]:
import os
import json

root = '/Users/mhendriksen/Desktop/repositories/MSCOCO-Flikcr30K_FG/Annotations'

files = os.listdir(root)

datasets = []
for file in files:
    tmp_path = os.path.join(root, file)
    with open(tmp_path, 'rb') as f:
        data = json.load(f)
        datasets.append(data)
    print('Loaded dataset from ', tmp_path)

Loaded dataset from  /Users/mhendriksen/Desktop/repositories/MSCOCO-Flikcr30K_FG/Annotations/Flickr30K_FG_ann.json
Loaded dataset from  /Users/mhendriksen/Desktop/repositories/MSCOCO-Flikcr30K_FG/Annotations/MSCOCO_FG_ann.json


In [72]:
f30k_ds_aug, coco_ds_aug = datasets[0], datasets[1]

In [76]:
from src.data.dataset import Dataset
from src.utils.utils import get_config
from src.utils.dataset_preprocessing import load_json_annotations

dataset = 'f30k'
model_name = 'clip'

config = get_config(dataset=dataset, model=model_name)

json_file = load_json_annotations(config=config)

f30k_dataset = Dataset(config=config, split='test', json_file=json_file)

Config path:  ./config/clip/f30k/development_local.yaml
Loaded config path
Loaded annotations from  /Users/mhendriksen/Desktop/repositories/datasets/f30k/annotations/dataset_flickr30k.json


In [107]:
dataset = 'coco'
model_name = 'clip'

config = get_config(dataset=dataset, model=model_name)

json_file = load_json_annotations(config=config)

coco_dataset = Dataset(config=config, split='test', json_file=json_file)

Config path:  ./config/clip/coco/development_local.yaml
Loaded config path
Loaded annotations from  /Users/mhendriksen/Desktop/repositories/datasets/coco/annotations/dataset_coco.json


In [122]:
json_file['images']

[{'filepath': 'val2014',
  'sentids': [770337, 771687, 772707, 776154, 781998],
  'filename': 'COCO_val2014_000000391895.jpg',
  'imgid': 0,
  'split': 'test',
  'sentences': [{'tokens': ['a',
     'man',
     'with',
     'a',
     'red',
     'helmet',
     'on',
     'a',
     'small',
     'moped',
     'on',
     'a',
     'dirt',
     'road'],
    'raw': 'A man with a red helmet on a small moped on a dirt road. ',
    'imgid': 0,
    'sentid': 770337},
   {'tokens': ['man',
     'riding',
     'a',
     'motor',
     'bike',
     'on',
     'a',
     'dirt',
     'road',
     'on',
     'the',
     'countryside'],
    'raw': 'Man riding a motor bike on a dirt road on the countryside.',
    'imgid': 0,
    'sentid': 771687},
   {'tokens': ['a',
     'man',
     'riding',
     'on',
     'the',
     'back',
     'of',
     'a',
     'motorcycle'],
    'raw': 'A man riding on the back of a motorcycle.',
    'imgid': 0,
    'sentid': 772707},
   {'tokens': ['a',
     'dirt',
     'pa

In [77]:
from tqdm import tqdm

for key in tqdm(f30k_dataset.images.keys()):
    # print(f30k_dataset.images[key]['sentids'])
    filename = f30k_dataset.images[key]['filename'].split('.')[0]
    sentids = f30k_dataset.images[key]['sentids']
    # print('filename: ', filename)
    augmented_captions = f30k_ds_aug[filename]
    # print('augmented_captions: ', augmented_captions)
    for aug_caption, sentid in zip(augmented_captions, sentids):
        f30k_dataset.update_caption(caption_idx=sentid, new_caption=aug_caption)

100%|██████████| 1000/1000 [00:00<00:00, 259628.85it/s]


In [114]:
path = 'annotations/dataset_coco.json'

path

'annotations/dataset_coco.json'

In [92]:
from tqdm import tqdm

def update_ds(dataset, augmented_dataset):

    for key in tqdm(dataset.images.keys()):
        # print(f30k_dataset.images[key]['sentids'])
        filename = dataset.images[key]['filename'].split('.')[0]
        sentids = dataset.images[key]['sentids']
        # print('filename: ', filename)
        augmented_captions = augmented_dataset[filename]
        # print('augmented_captions: ', augmented_captions)
        for aug_caption, sentid in zip(augmented_captions, sentids):
            dataset.update_caption(caption_idx=sentid, new_caption=aug_caption)

    return dataset
    
f30k_dataset_augmented = update_ds(dataset=f30k_dataset, augmented_dataset=f30k_ds_aug)

100%|██████████| 1000/1000 [00:00<00:00, 375699.03it/s]


In [93]:
coco_dataset_augmented = update_ds(dataset=coco_dataset, augmented_dataset=coco_ds_aug)

100%|██████████| 5000/5000 [00:00<00:00, 328557.87it/s]


In [112]:
path = '/Users/mhendriksen/Desktop/repositories/evaluating-cmr-in-mm/src/data/augmented'

with open

('a black bird in a pot eating a fruit',
 <PIL.Image.Image image mode=RGB size=640x481>,
 97009,
 228,
 'COCO_val2014_000000208589.jpg',
 155)

In [113]:
coco_dataset[155]

('A bird in a pot eating a fruit.',
 <PIL.Image.Image image mode=RGB size=640x481>,
 97009,
 228,
 'COCO_val2014_000000208589.jpg',
 155)

In [103]:
coco_ds_aug['COCO_val2014_000000561100']

['a small blue plane sitting on top of a field',
 'an e2 airplane painted blue with black and white stripes',
 'an old warplane is on display in a field',
 'a blue small plane standing at the airstrip',
 'model airplane with an american insignia and stripes on wings']

In [85]:
print(test.captions[125]['raw'])
print(test.captions[126]['raw'])
print(test.captions[127]['raw'])
print(test.captions[128]['raw'])
print(test.captions[129]['raw'])

the man with pierced ears is wearing glasses and an orange hat on his head
a man with glasses is wearing a beer can crocheted hat
a man with pierced ears and glasses is wearing an orange hat on his head
a man in an orange hat starring at something
a man wears an orange hat and glasses


In [84]:
print(f30k_dataset.captions[125]['raw'])
print(f30k_dataset.captions[126]['raw'])
print(f30k_dataset.captions[127]['raw'])
print(f30k_dataset.captions[128]['raw'])
print(f30k_dataset.captions[129]['raw'])

the man with pierced ears is wearing glasses and an orange hat on his head
a man with glasses is wearing a beer can crocheted hat
a man with pierced ears and glasses is wearing an orange hat on his head
a man in an orange hat starring at something
a man wears an orange hat and glasses


In [55]:
print(f30k_dataset.captions[125]['raw'])
print(f30k_dataset.captions[126]['raw'])
print(f30k_dataset.captions[127]['raw'])
print(f30k_dataset.captions[128]['raw'])
print(f30k_dataset.captions[129]['raw'])

the man with pierced ears is wearing glasses and an orange hat on his head
a man with glasses is wearing a beer can crocheted hat
a man with pierced ears and glasses is wearing an orange hat on his head
a man in an orange hat starring at something
a man wears an orange hat and glasses


In [31]:
f30k_dataset.images[25]

{'filename': '1007129816.jpg', 'sentids': [125, 126, 127, 128, 129]}